# Input module examples

This is not yet perfect, but is getting there. The dataframes which are returned are designed to reflect MAGICC's underlying data input format. When we have pint with units (waiting on https://github.com/hgrecco/pint/pull/684) we can use `df.quantify()` and `df.dequantify()` to quickly add/remove units to our dataframes and swiftly do conversions etc. 

In [1]:
from os.path import join
from pprint import pprint

import pandas as pd
import pymagicc
from pymagicc.input import MAGICCInput

import expectexception

/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
MAGICC6_DIR = join("..", "pymagicc", "MAGICC6", "run")

In [3]:
"""
temporary sandbox: delete later

DELETE
DELETE
DELETEDELETEDELETE
DELETE
DELETE
DELETE
DELETE
DELETE
"""
mdata = MAGICCInput()
mdata.read(MAGICC6_DIR, "MIXED_NOXI_OT.IN")
mdata.df

AssertionError: Expected a header token of GAS, got YEARS

## Read files

In [ ]:
mdata = MAGICCInput()
mdata.read(MAGICC6_DIR, "HISTRCP_CO2I_EMIS.IN")
mdata.head()

In [ ]:
pprint(mdata.metadata)

## Accessors

`MAGICCInput` is built to make data accessing as simple as possible. As we have unique keys for each of our headers, we allow users to select data simply using lists, without specifying what each value should correspond to etc. (essentially we hide away all the pandas fiddly stuff).

Of course the underlying dataframe is always accessible via the `.df` attribute.

In [ ]:
mdata['CO2I_EMIS'].head(2)

In [ ]:
mdata['R5ASIA', 'CO2I_EMIS'].head(2)

In [ ]:
# changing order doesn't matter
mdata['CO2I_EMIS', 'R5ASIA'].head(2)

In [ ]:
%%expect_exception KeyError
# the above doesn't work on the raw dataframe
# because you need to specify all the levels 
# in the right order
mdata.df['R5ASIA', 'CO2I_EMIS']

In [ ]:
# can also put index values in
mdata[1956, 'CO2I_EMIS', 'R5ASIA'].head(2)

In [ ]:
%%expect_exception KeyError
# although slicing fails, one for the to do list...
mdata[1956:1970, 'CO2I_EMIS', 'R5ASIA'].head(2)

In [ ]:
mdata[1998]

### Reading SCEN files

We can also read SCEN files. These are particularly tricky as the underlying data format is unique.

In [ ]:
mdata.read(MAGICC6_DIR, "RCP45.SCEN")
mdata.df.head()

In [ ]:
mdata["BC", "R5ASIA"].head()

## Writing files

Once you have your data in the format as above, writing files is trivial. 

In [ ]:
mdata.read(MAGICC6_DIR, "HISTRCP_CO2I_EMIS.IN")
mdata.write("HISTEXAMPLE_CO2I_EMIS.IN")

mdata.read(MAGICC6_DIR, "RCP45.SCEN")
mdata.write("SCENEXAMPLE.SCEN", filepath='.')

However, note that the format to write in is determined by the filename. The rules for this are not yet documented (TODO!) but the errors are fairly clear. Hence you can't just any old filename, it has to follow MAGICC's (somewhat cryptic) internal conventions.

In [ ]:
%%expect_exception ValueError
mdata.write("histexample.txt")